<table align="center">
   <td align="center"><a target="_blank" href="https://colab.research.google.com/github/umbcdata602/fall2020/blob/master/lab_backpropagation.ipynb">
<img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
</table>

# Lab -- Backpropagation

The goal here is to develop an intuition for backpropagation -- with minimal math.



# Supersimplified case

The supersimplified model is

$$
\hat{y} = wx
$$

* $x$ the model input (feature)
* $\hat{y}$ is the model prediction of the target variable $y$
* $w$ is the trainable weight

Let $\epsilon = \hat{y} - y$ represent the residual of the model prediction. 

The cost function $J$ is
$$
J = \epsilon^2 = (y - wx)^2
$$
 
With multiple input features and samples, then $J$ involves multiple sums, which make the mathematical manipulations more complicated. With neural networks, there are multiple layers of calcluations, which makes things worse. We'll keep it simple.

## The solution

$J$ is a minimum for 

$$
\frac{\partial J}{\partial w} = 0
$$

The gradient of $J$ with respect to the weight $w$ is

$$
\frac{\partial J}{\partial w} = 2(y-wx)(-x) = -2x(y-wx) = -2x\epsilon
$$

That is,

$$\epsilon = y - wx = 0$$

In this supersimplified case, the solution is obvious by inspection

$$
w = \frac{y}{x}
$$

The model input is $x$, and $y$ is the target output.

# Slightly more complex case

We'll add some complexity by introducing some nonlinearity, and perform the calculations in a sequence of "Layers."

* Layer 1 multiplies the weight $w$ by the input $x$. The output of Layer 1 is the resulting product $z$,

$$
\mathrm{Layer~1:~~} z = wx
$$

* Layer 2 takes the output of Layer 1 and computes a nonlinear function $f(z)$. The output of Layer 2 is a prediction $\hat{y}$ of $y$:

$$
\mathrm{Layer~2:~~} \hat{y} = f(z)
$$

* The cost function $J$ can be computed with the result of Layer 2.

$$
J = (y - f(z))^2 = \epsilon^2
$$

* The derivative of $J$ wrt the weight $w$ is

$$
\frac{\partial J}{\partial w} = 
2(y-f(z))\left( - \frac{\partial f}{\partial z} \frac{\partial z}{\partial w} \right) = -2 \epsilon \frac{\partial f}{\partial z} \frac{\partial z}{\partial w}
$$

## Gradient descent (review)

The gradient-descent algorithm is based on the fact that, near $\mathbf{x}$, $J(\mathbf{x})$ decreases fastest if you go in the direction of $- \nabla J(\mathbf{x})$. The following figure, which comes from the wikipedia entry for [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent), shows contours of $J$ as a function of $\mathbf{x}$.

<img src="https://upload.wikimedia.org/wikipedia/commons/f/ff/Gradient_descent.svg" width="250"/>

$J$ is a minimum in the center. The gradient-descent algorithm involves updates to $\mathbf{x}$ in that direction:

$$ \mathbf{x}_{i+1} = \mathbf{x}_i - \eta \nabla J(\mathbf{x_i}) $$

$\eta$ is the learning rate.

## Gradient descent (our case)

* We want updates to weights $w$ in the direction of $- \nabla J = - \frac{\partial J}{\partial w}$

$$
w_{i+1} = w_i - \eta \frac{\partial J}{\partial w}
$$

* We need the outputs from each layer in order to compute $\nabla J$:

$$
\frac{\partial J}{\partial w} = 
 -2 \epsilon \frac{\partial f}{\partial z} \frac{\partial z}{\partial w}
$$

The algorithm:

* Initialize $w_i$
$$
\mathrm{Initialization~:~~} w_i = \mathrm{random~number}
$$
* Use $w_i$ to compute $\hat{y}_i$
$$
\mathrm{Forward~Propagation:~~} \epsilon
$$
* Backpropagation to Layer 2
$$
\mathrm{Layer~2:~~} \epsilon \frac{\partial f}{\partial z} 
$$
* Backpropagation to Layer 1
$$
\mathrm{Layer~1:~~} \epsilon \frac{\partial f}{\partial z} \frac{\partial z}{\partial w}
$$
* Update the weights

$$
w_{i+1} = w_i - \eta \epsilon \frac{\partial f}{\partial z} \frac{\partial z}{\partial w}
$$

* Go back to Forward Propagation with these new weights, and repeat until done.


Here are the considerations that generalize this layered approach to multilayer neural networks and make backpropagation so powerful:
* With neural networks, there are many layers and each layer has its own set of weights $w$ and nonlinear activation functions $f(x)$, so $\nabla J$ involves many matrix-matrix multiplications (Raschka mentions the Jacobians). Not only do these multiple matrix multiplications make the mathematics complex, they are computationally expensive and troublesome.
* The order of the multiplications matters, big time. Backpropagation takes advantage of the fact that the last layer in the network involves multiplication by $\epsilon$, which is a vector (1-D array). By starting at the end and working backward through the network, calculation of $\nabla J$ begins with a matrix-vector multiplication, which produces another vector. Each subsequent backward step through the network likewise involves only matrix-vector multiplications. Each of these steps is much easier than the matrix-matrix multiplication that would be involved in the forward direction -- for big neural networks the savings is huge.
* With backpropagation, the backward pass is comparable to the foward pass in terms of computational expense. And as long as you can take derivatives of the cost function and activation functions, then the backpropagation step can implemented behind the scenes with automatic differentiation.
* $J$ is bumpy. With Adaline, $f(z)$ is linear, $J$ is convex, and the solution occurs where $\frac{\partial J}{\partial w} = 0$. But with neural networks, the output of each layer involves nonlinear activation functions that conspire to give $J$ complex structure, big time.
* Since $J$ is generally "bumpy" (the figure below is from Raschka's [ch12.ipynb](https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch12/ch12.ipynb)), "noisy" forms of gradient descent become our friend.


<img src="https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch12/images/12_13.png" width="600"/>

* With batch gradient descent, we risk getting caught in a local minima. That's where the "noisy" behavior of stochastic gradient descent can help. As the algorithm progresses down the gradient in search of a global minimum of $J$, SGD can knock the algorithm out of local minima.
* Backpropagation needs various things that are computed during forward propagation (e.g., $\frac{\partial f}{\partial z}$ and $\frac{\partial z}{\partial w}$). In practice, the algorithm keeps track of these intermediate products. As we'll see with Tensorflow, that's the purpose of GradientTape.
